We want to start of by saying that this notebook is kind of all over the place, since 4 people, many of which have little coding experience. Apart from a few functions, the code is not well documented. This should have happends during the development. Never the less, we quickly added some comments above the function to guide with explaining their purpose. The code is also not really finished as there are still a lot of debug statements in the code

Note. all nessecary dependencies should be installed as we do not do that in the code.

In [ ]:
#import all depencencies
import pathlib
import sys

import jupedsim as jps
import matplotlib.pyplot as plt
import pandas as pd
import pedpy
from numpy.random import normal  # normal distribution of free movement speed
from shapely import GeometryCollection, Polygon
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
from shapely.geometry import Polygon, GeometryCollection

import numpy as np
import os
from datetime import datetime, timedelta
import sqlite3
from shapely.affinity import translate
import random


from sociophysicsDataHandler import SociophysicsDataHandler
import plot_basics as pb

In [ ]:
#Return data from a full hour, return error if there is none.
def fetchhour(dh, day, hour):
    """
    Fetches all the data from an hour given a certain day.

    Parameters:
    - df: DataFrame with columns ['utc', 'tracked_object', 'x_pos', 'y_pos']
    - day: day of the requested data in 'yyyymmdd' format
    - hour: hour of wanted data

    Returns:
    - df: dataframe with the data of asked hour and day
    """

    GEN_ENTRY_L1 = "ehv/platform2.1/"
    GEN_ENTRY_L2 = '/EHV_Platform2.1_'
    GEN_ENTRY_L3 = '_trajectorie.parquet' 
    path = GEN_ENTRY_L1 + day + GEN_ENTRY_L2 + day + hour + GEN_ENTRY_L3
    
    try:
        dh.fetch_prorail_data_from_path(path, verbose=False)
    except HTTPResponseError as e:
        print(f"Data not found for {day} hour {hour} (404)")
        return None

    return dh.df

#Fetch data from a day, filter on position and train arrival. Can be enabled/disabled
def fetchday(dh, day, trains, filter_pos=True, filter_time=True):
    """
    Fetches and combines data for all 24 hours of a given day.

    Parameters:
    - dh: Data handler object with method `fetch_prorail_data_from_path`
    - day: Day of the requested data in 'yyyymmdd' format
    - trains: dataframe with the train information
    - filter_pos: option to filter position of platform
    - filter_time: option to filter time arrival trains

    Returns:
    - df_all: Combined DataFrame with all hourly data found for that day
    """
    
    all_dfs = []

    for hour in range(24):
        hour_str = f"{hour:02d}"  
        df = fetchhour(dh, day, hour_str)
        if df is not None:
            all_dfs.append(df)

    if not all_dfs:
        print(f"No data collected for {day}")
        return pd.DataFrame()


   
    dh.df = pd.concat(all_dfs, ignore_index=True)
    dh.df = dh.df.sort_values(by=['tracked_object', 'date_time_utc'])
    
    #Remove data the is not from the lower part of the platform
    if filter_pos:
        min_x = -2650
        dh.df = dh.df[dh.df['x_pos'] <= min_x] 
    
    
    #Only ensure data in the dataset from when train arrives
    if filter_time:
        
        date = pd.to_datetime(day, format = "%Y%m%d")
        trains['date'] = pd.to_datetime(trains['date'])
        train = trains[(trains['date'] == date)]

        if not train.empty:
            subsets = []
            subsets2 = []
            train = train[train['door_x'].notna()]
            for arrival in train['arrival_time'].unique():
                start = arrival
                end = start + pd.Timedelta(minutes=1)
                
                subset = dh.df[(dh.df['datetime']  == start)]
                subsets.append(subset)

                subset2 = dh.df[(dh.df['datetime']  >= start) & (dh.df['datetime'] <= end)]
                subsets2.append(subset2)
                
                
            dh.df = pd.concat(subsets) if subsets else pd.DataFrame(columns=dh.df.columns)
            dh.df2 = pd.concat(subsets2) if subsets2 else pd.DataFrame(columns=dh.df.columns)
        else:
            if train.empty:
                print("No trains found for that date")
            if train['door_x'].isna().any():
                print("x_door is Nan")
            
            dh.df.drop(dh.df.index, inplace=True)
            dh.df.head()
        
        dh.df = dh.df.sort_values(by=['tracked_object', 'date_time_utc'])

    dh.df = dh.df.reset_index(drop=True)
    return dh.df, dh.df2

#Generate model parameters
def generate_parameters(current_crowdedness=60, average_crowdedness=28):
    crowdedness_ratio = (current_crowdedness - average_crowdedness) / average_crowdedness

    crowded_weight = 0.1  # The weight of the crowdedness effect compared to the mean
    anticipation_time = float(1.0)
    reaction_time = 0.25

    wall_buffer_distance = float(np.clip(np.random.normal(0.1, 0.01) + 0.1 * crowded_weight * crowdedness_ratio, 0.05, 0.15))
    time_gap = float(np.clip(np.random.normal(1.06, 0.106) + 1.06 * crowded_weight * crowdedness_ratio, 0.53, 1.59))
    range_neighbor_repulsion = float(np.clip(np.random.normal(4, 0.01) + 0.1 * crowded_weight * crowdedness_ratio, 0.05, 5))
    strength_neighbor_repulsion = float(np.clip(np.random.normal(2, 0.8) + 8 * crowded_weight * crowdedness_ratio, 4, 12))

    return {
        "wall_buffer_distance": wall_buffer_distance,
        "anticipation_time": anticipation_time,
        "reaction_time": reaction_time,
        "time_gap": time_gap,
        "range_neighbor_repulsion": range_neighbor_repulsion,
        "strength_neighbor_repulsion": strength_neighbor_repulsion,
    }

#Generate the map, with dynamic door allocation
def generate_doors(train, time, door_thick=1, door_height=1.5):
    # Define the sloped left and right wall edges
    left_wall_start = (8.5, -5)
    left_wall_end = (9.5, 74)
    right_wall_start = (9.5, -5)
    right_wall_end = (10.5, 74)

    # Interpolation helper(WHY IN THIS FUNCTION???)
    def interpolate_x(x0, y0, x1, y1, y):
        return x0 + ((y - y0) / (y1 - y0)) * (x1 - x0)

    # Filter data for the given time
    time_filtered = train[train['arrival_time'] == time]
    
    door_polygons = []

    for _, row in time_filtered.iterrows():
        center_y = row["door_x"] / 1000  # Convert mm to meters

        y1 = center_y - door_height / 2
        y2 = center_y + door_height / 2

        # Get corresponding x-values from sloped walls
        x1_left = interpolate_x(*left_wall_start, *left_wall_end, y1)
        x2_left = interpolate_x(*left_wall_start, *left_wall_end, y2)

        x1_right = interpolate_x(*right_wall_start, *right_wall_end, y1)
        x2_right = interpolate_x(*right_wall_start, *right_wall_end, y2)

        # Define a trapezoidal "door"
        door = Polygon([
            (x1_left, y1),
            (x1_right, y1),
            (x2_right, y2),
            (x2_left, y2)
        ])
        door_polygons.append(door)

    all_doors = unary_union(door_polygons)

    # Define left and right rooms
    left_room = Polygon([(-4, -5), (8.5, -5), (9.5, 74), (-3, 74)])
    right_room = Polygon([(9.5, -5), (11.5, -5), (12.5, 74), (10.5, 74)])

    # Define obstacles (same as before)
    shop = Polygon([(4.5, 28.5), (1, 28.5), (1, 49.5), (4.5, 49.5)]) 
    blob = Polygon([(5, 72), (1, 72), (1, 74), (5, 74)])   
    left_inner_wall = Polygon([(6, -5), (6, 4), (5.5, 4), (5.5, -5)]) 
    right_inner_wall = Polygon([(-1.5, -5), (-1.5, 4), (-1, 4), (-1, -5)]) 

    obstacles = shop.union(blob).union(left_inner_wall).union(right_inner_wall)

    # Subtract obstacles from left room
    left_room_with_holes = left_room.difference(obstacles)

    # Combine geometry
    combined_geometry = left_room_with_holes.union(right_room).union(all_doors)

    walkable_area = pedpy.WalkableArea(combined_geometry)

    # Plot Commeted out as of 11/06 because it keeps printing for some reasons
    #ax = pedpy.plot_walkable_area(walkable_area=walkable_area)
    #ax.set_aspect("equal")

    # Exit areas (translated forward 1.5m)
    exit_areas = [translate(door, xoff=1.5) for door in door_polygons]

    

    return walkable_area, exit_areas, combined_geometry

#Generate spawn location for agents
def generate_ped_spawn(ped_spawn, time, radius = 0.25):
    ped = ped_spawn[ped_spawn['datetime'] == time].copy()
    ped['x_pos'] = (-ped['x_pos']) / 1000 #right scaling and conversion to meter
    ped['y_pos'] = ped['y_pos'] / 1000 #Convert to meter

    spawn_polygon = []
    for _, row in ped.iterrows():
        spawn_area = Point(row['x_pos'], row['y_pos']).buffer(radius)
        spawn_polygon.append(spawn_area)
        
    return spawn_polygon

#Spawn agents in spawn location
def generate_agent_loc(spawn_polygons):
    return [poly.centroid for poly in spawn_polygons]

#Generate map with agents
def plot_simulation_configuration(
    walkable_area, spawning_area, starting_positions, exit_areas
):
    axes = pedpy.plot_walkable_area(walkable_area=walkable_area)

    for area in spawning_area:
        axes.fill(*area.exterior.xy, color="lightgrey")
    
    for exit_area in exit_areas:
        axes.fill(*exit_area.exterior.xy, color="indianred")
        
    coords = [(p.x, p.y) for p in starting_positions]
    axes.scatter(*zip(*coords), s=1)
    axes.set_xlabel("x/m")
    axes.set_ylabel("y/m")
    axes.set_aspect("equal")

#Create the SQLite file
def create_file(area, exitareaPed):

# Save explicitly to the notebook's directory
    trajectory_file = os.path.join(os.getcwd(), f"bottleneck_{datetime.now().strftime('%Y%m%d_%H%M%S')}.sqlite")
    trajectory_path = pathlib.Path(trajectory_file)

    area = area.buffer(0)
    try:
        simulation = jps.Simulation(
            model=jps.AnticipationVelocityModel(
            #pushout_strength=0.3
            ),
            geometry=area,
            trajectory_writer=jps.SqliteTrajectoryWriter(
                output_file=pathlib.Path(trajectory_file)
            ),
            timestep=0.1,
        )
    except RuntimeError as e:
        print("Simulation creation failed due to invalid geometry:", e)
        return None, None, None, None, None
    
    os.makedirs(trajectory_path.parent, exist_ok=True)

    print(f"Saving trajectory to: {trajectory_file}")

    try:
        exit_id1 = simulation.add_exit_stage(exitareaPed[0].exterior.coords[:-1])
        exit_id2 = simulation.add_exit_stage(exitareaPed[1].exterior.coords[:-1])
        exit_id3 = simulation.add_exit_stage(exitareaPed[2].exterior.coords[:-1])
        exit_id4 = simulation.add_exit_stage(exitareaPed[3].exterior.coords[:-1])
        exit_id5 = simulation.add_exit_stage(exitareaPed[4].exterior.coords[:-1])
    except Exception as e:
        print(f"[Warning] Could not add exit a stage: {e}")
        return None, None, None, None, None
    
        
    exit_idsleft = [exit_id1, exit_id2, exit_id3, exit_id4, exit_id5]

    journey_left = jps.JourneyDescription([exit_id1, exit_id2, exit_id3, exit_id4, exit_id5])
    journey_id_left = simulation.add_journey(journey_left)

    

    return simulation, trajectory_path, exit_idsleft, journey_id_left, trajectory_file


#Spawn the agents on the map with the right model parameters and assign exit loc
def run_sim(simulation, agents_spawn, exitareaPed, exit_idsleft, journey_id_left, time, ped_spawn ,prev_agent_count ,agent_radius = 0.18):
    num_agents_left = len(ped_spawn[ped_spawn['datetime']== time])
    v_distribution_left = normal(1.34, 0.05, num_agents_left)
    #print(v_distribution_left)
    error_add = 0
    
    exit_areasleft = [exitareaPed[0], exitareaPed[1], exitareaPed[2], exitareaPed[3], exitareaPed[4]]


    #exit_idsleft = [exit_id1, exit_id2, exit_id3, exit_id4, exit_id5]
    #exit_idsright = [exit_id6, exit_id7, exit_id8, exit_id9, exit_id10]

    exit_centroids_left = [ea.centroid for ea in exit_areasleft]



    ped = ped_spawn[ped_spawn['datetime'] == time].copy().reset_index(drop=True)
    id_mapping = []
    
    count = 0
    for _ in zip(agents_spawn):
        count +=1

    for pos, v0, original_id in zip(agents_spawn, v_distribution_left, ped['tracked_object']):
        try:
            agent_point = Point(pos)
            closest_exit_idx = min(
                range(len(exit_centroids_left)),
                key=lambda i: agent_point.distance(exit_centroids_left[i])
            )
            parameter_dict = generate_parameters(current_crowdedness= count)

    # This code will set the radius based on whether the agent is male or female
            if (random.randint(0,1) == 0):
            # Female condition
                radius= float(np.clip(np.random.normal(loc=0.419, scale=0.035), 0.373, 0.539)/2)
            # 0.469 m is the mean arm-to-spine width (radius), with an sd of 0.035 m, the maximum 0.609 m and the minimum 37.3 m
            else:
        # Male condition
                radius= float(np.clip(np.random.normal(loc=0.546, scale=0.0436), 0.399, 0.636)/2)
        # 0.546 m is the mean arm-to-spine width (radius), with an sd of 0.0436 m, the maximum  0.725 m and the minimum 0.399 m
        # https://multisite.eos.ncsu.edu/www-ergocenter-ncsu-edu/wp-content/uploads/sites/18/2016/06/Anthropometric-Detailed-Data-Tables.pdf
        
            simulation.add_agent(
                jps.AnticipationVelocityModelAgentParameters(
                    journey_id=journey_id_left,
                    stage_id=exit_idsleft[closest_exit_idx],
                    position=(pos.x, pos.y),
                    desired_speed=v0,
                    radius = radius,
                    **parameter_dict
                )
            )
            id_mapping.append({'original_id': original_id, 'agent_id': prev_agent_count + simulation.agent_count() + error_add})
            print(f"Succes -> Original ID: {original_id} Simulation ID: {simulation.agent_count()}")
            
        except RuntimeError as e:
            error_add +=1
            print(f"{e}")

    print("Total agents added:", simulation.agent_count())
    print(id_mapping)

    return id_mapping, simulation.agent_count(), error_add

#Run the sim per timestep and gether the position data
def data_gether(simulation, start_time, prev_agent_count, error_add, max_step = 45000):
    step =0

    current_time= 0
    data = []
    count = simulation.agent_count()
    print(count)

    if not isinstance(start_time, str):
        start_time = str(start_time)
    
    utc_start_time = datetime.fromisoformat(start_time)

    ######################
    
    print(dir(simulation))
    agents = list(simulation.agents())
    agent_ids = [agent.id for agent in agents]
    print("Active agent IDs:", agent_ids)


    ###########################

    while simulation.agent_count() > 0 and step < max_step:
        #print("test")
        simulation.iterate()
        if step % 10 == 0:
            current_time = step * 0.01
            #print(f"current time: {current_time}")

            time_add = utc_start_time + timedelta(seconds=step * 0.01)
            time_add_str = time_add.isoformat()

            for i in range(prev_agent_count, prev_agent_count+count+error_add):
                try:
                    #print(f"Trying agent:{i+1}")
                    agent = simulation.agent(i+1)
                    data.append({
                        'time': time_add_str,
                        'agent_id': agent.id,
                        'x_pos': agent.position[0],
                        'y_pos': agent.position[1]
                    })    
            
                except RuntimeError as e:
                    #print(f"Simulation error: {e}")
                    continue
            

        step += 1
        
    print(f"Agents still active: {simulation.agent_count()}")
    not_finished = 0;
    if simulation.agent_count() > 0:
        not_finished = 1;

    for i in range(count):
        try:
            agent = simulation.agent(i)
            #print(f"Agent {agent.id} at {agent.position}, stage_id: {agent.stage_id}")
        except RuntimeError as e:
            print("Agent dead")
    return data, not_finished

#Change the simulation ID to the actual ID from the data
def merge_data(ids_table, pred_data_unmerged):
    df = pd.DataFrame(pred_data_unmerged)
    ids = pd.DataFrame(ids_table)

    print(type(df))
    print(type(ids))

    print(df.head())
    print(ids.head())
    
    pred_data_merged = pd.merge(df, ids[['agent_id', 'original_id']], on='agent_id', how='left')

    pred_data_merged = pred_data_merged.rename(columns={'original_id': 'tracked_object'})
    pred_data_merged = pred_data_merged.rename(columns={'time': 'datetime'})

    return pred_data_merged

#The finale pipelined function to run a single time instance 
def run_single_instance(trains, time, date, ped_spawn, total_agents = 0 ,not_print = True):
    try:
        walkable_area, exitareaPed, area = generate_doors(trains, time)
    except Exception as e:
        print(f"[Warning] Geometry error at time index {time}: {e}")
        print(time)
        return pd.DataFrame()
    spawn_poly = generate_ped_spawn(ped_spawn, time)
    agents_spawn = generate_agent_loc(spawn_poly)
    simulation, trajectory_path, exit_idsleft, journey_id_left, trajectory_file = create_file(area, exitareaPed)
    if simulation is None:
        return pd.DataFrame(), total_agents, trajectory_file
        
    ids_table, agent_count, error_add= run_sim(simulation, agents_spawn, exitareaPed, exit_idsleft, journey_id_left, time, ped_spawn, total_agents)
    pred_data_unmerged, not_finished = data_gether(simulation, time, total_agents, error_add)
    final_prediction = merge_data(ids_table, pred_data_unmerged)

    current_agent_count = total_agents + agent_count + error_add
    
    return final_prediction, current_agent_count, trajectory_file, not_finished

#Calculate the ADE for a single senario
def calculate_ade(prediction, truth):
    gt = truth.copy()
    pred = prediction.copy()

    gt['datetime'] = pd.to_datetime(gt['datetime'],format='ISO8601', utc = True)
    pred['datetime'] = pd.to_datetime(pred['datetime'], format='ISO8601',utc = True)

    merged = pd.merge(gt, pred, on=['tracked_object', 'datetime'], suffixes=('_gt', '_pred'))

    print("Merged Columns:", merged.columns)
    
    merged['displacement'] = np.sqrt(
        (merged['x_pos_gt'] - merged['x_pos_pred']) ** 2 +
        (merged['y_pos_gt'] - merged['y_pos_pred']) ** 2
    )

    ade = merged['displacement'].mean()
    return ade

#Calculate ADE from NN dataframe(obsolete)
def calculate_ade_from_single_df(df):
    df = df.copy()
    
    df['displacement'] = np.sqrt(
        (df['true_x'] - df['curr_x']) ** 2 +
        (df['true_y'] - df['curr_y']) ** 2
    )
    
    ade = df['displacement'].mean()
    return ade

#Convert NN DF to the right format(obsolete)
def convert_to(file_name):
    df = pd.read_csv(file_name)
    df = df.drop(columns=['node_type', 'pred_x', 'pred_y', 'error'])
    df['curr_x'] = df['curr_x']/1000
    df['curr_y'] = df['curr_y']/1000
    df['true_x'] = df['true_x']/1000
    df['true_y'] = df['true_y']/1000

    return df

#Calculate the ADE for the NN 
def calculate_ade_over_time_steps(file_names):
    ades = []
    for file in file_names:
        df = convert_to(file)
        ade = calculate_ade_from_single_df(df)
        ades.append(ade)

    overall_ade = np.mean(ades)
    return overall_ade

#Funtion for some Data analysis, abandonned due to time constained!
def door_arrival(pred, ped, trains, datetime):
    pred = pred.sort_values(by='datetime')
    pred['last_post'] = pred.sort_values('datetime').groupby('tracked_object').tail(1).reset_index(drop=True) 

    df_last = pred.sort_values(by='datetime').groupby('tracked_object').tail(1)
    df_last = df_last[['tracked_object', 'x_pos', 'y_pos']].rename(
    columns={'x_pos': 'x_last', 'y_pos': 'y_last'})

    df_with_last = df.merge(df_last, on='tracked_object', how='left')

    df_with_last.head(20)
    train = trains[(trains['arrival_time'] == datetime)]
    train.head()

#Added prediction of NN in 1 file
def add_together_files(file_list):

    combined_df = pd.DataFrame()
    
    for idx, file in enumerate(file_list, start=1):
        df = pd.read_csv(file)
        df = df.drop(columns=['node_type', 'pred_x', 'pred_y', 'error'])
        df['datetime'] = idx
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    return combined_df
        
#Would take a day as input and return the ADE DF for a whole day, AKA THE FINAL BOSS
def calculate_ade_day(day):
    dh = pb.get_SociophysDH()
    dh.fetch_prorail_train_information()
    trains = dh.train_information
    ped_train_loc, ped_data = fetchday(dh, day, trains)
    times = ped_train_loc['datetime'].unique()
    not_finished = 0;

    if ped_train_loc.empty:
        print(f"No data for {day}")
        return pd.DataFrame()
        
    print("Fetched data")
    ades= []
    step = 0
    nr_agents = 0
    for time in times:
        print(f"Processing time step {step + 1} of {len(times)}, DateTime: {time}")
        step += 1

        old_nr_agents = nr_agents
        pred, nr_agents, traj_file, not_finished = run_single_instance(trains, time, day, ped_train_loc, nr_agents ,not_print= False)

        not_finished += not_finished
        simulation_nr_agents = nr_agents - old_nr_agents
        if pred.empty:
            ades.append({
            'time' : time,
            'ade'  : 0  
            })

        else:
            
            start_time = pd.to_datetime(pred['datetime'].iloc[0])
            end_time = start_time + pd.Timedelta(minutes=1)
        
            ped = ped_data[(ped_data['datetime'] >= start_time) & (ped_data['datetime'] <= end_time)].copy()
            ped['x_pos'] = (-ped['x_pos']) / 1000 #right scaling and conversion to meter
            ped['y_pos'] = ped['y_pos'] / 1000
            ade = calculate_ade(pred, ped)
            ades.append({
                'time' : time,
                'ade'  : ade,
                'nr_agents'  :  simulation_nr_agents,
                'trajectory_file'  : traj_file
            })
        
    print(f"Amount of simulations not finished {not_finished}")    
    return pd.DataFrame(ades)